In [10]:
from datetime import datetime, timedelta

class Material:
    def __init__(self, titulo, estado='disponible'):
        self.titulo = titulo
        self.estado = estado

    def prestar(self):
        if self.estado == 'disponible':
            self.estado = 'prestado'
            return True
        return False

    def devolver(self):
        self.estado = 'disponible'

    def __str__(self):
        return f"{self.titulo} ({self.estado})"


class Libro(Material):
    def __init__(self, titulo, autor, genero):
        super().__init__(titulo)
        self.autor = autor
        self.genero = genero

    def __str__(self):
        return f"Libro: {self.titulo} por {self.autor} ({self.genero})"

class Revista(Material):
    def __init__(self, titulo, edicion, periodicidad):
        super().__init__(titulo)
        self.edicion = edicion
        self.periodicidad = periodicidad

    def __str__(self):
        return f"Revista: {self.titulo}, edicion: {self.edicion} ({self.periodicidad})"

class MaterialDigital(Material):
    def __init__(self, titulo, tipo_archivo, enlace):
        super().__init__(titulo)
        self.tipo_archivo = tipo_archivo
        self.enlace = enlace

    def __str__(self):
        return f"Material digital: {self.titulo} ({self.tipo_archivo}) enlace: {self.enlace}"

class Persona:
    def __init__(self, nombre, apellido):
        self.nombre = nombre
        self.apellido = apellido

class Usuario(Persona):
    def __init__(self,nombre,apellido):
        super().__init__(nombre,apellido)
        self.materiales_prestados = []
        self.penalizaciones = 0

    def pedir_prestado(self,material,fecha_prestamo,dias_prestamo=7):
        if material.prestar():
            fecha_devolucion = fecha_prestamo + timedelta(days=dias_prestamo)
            self.materiales_prestados.append((material, fecha_prestamo, fecha_devolucion))
            print(f"{self.nombre} pidio prestado '{material.titulo}' el {fecha_prestamo.strftime('%Y-%m-%d')}, debe devolverlo antes del {fecha_devolucion.strftime('%Y-%m-%d')}")
            return True
        print(f"No se pudo prestar '{material.titulo}', no esta disponible.")
        return False

    def devolver_material(self, material):
        for i, (j, fecha_prestamo, fecha_devolucion) in enumerate(self.materiales_prestados):
            if j == material:
                if datetime.now() > fecha_devolucion:
                    dias_retraso = (datetime.now() - fecha_devolucion).days
                    self.penalizaciones += dias_retraso
                    print(f"{self.nombre} ha devuelto '{material.titulo}' con {dias_retraso} dias de retraso. Penalizaciones acumuladas: {self.penalizaciones}")
                else:
                    print(f"{self.nombre} ha devuelto '{material.titulo}' a tiempo.")
                material.devolver()
                self.materiales_prestados.pop(i)
                return True
        print(f"'{material.titulo}' no esta en la lista de materiales prestados de {self.nombre}.")
        return False

class Bibliotecario(Persona):
    def __init__(self,nombre,apellido, sucursal):
        super().__init__(nombre,apellido)
        self.sucursal = sucursal

    def agregar_material(self, material):
        self.sucursal.agregar_material(material)

    def transferir_material(self, material, otra_sucursal):
        if self.sucursal.eliminar_material(material):
            otra_sucursal.agregar_material(material)
            print(f'{self.nombre} transfirio "{material.titulo}" a "{otra_sucursal.nombre}')

class Sucursal:
    def __init__(self, nombre):
        self.nombre = nombre
        self.catalogo = []

    def agregar_material(self, material):
        self.catalogo.append(material)
        print(f'{material.titulo} fue agregado a la biblioteca {self.nombre}') 

    def eliminar_material(self, material):
        if material in self.catalogo:
            self.catalogo.remove(material)
            print(f'El {material.titulo} fue eliminado de la biblioteca {self.nombre}')
            return True
        print(f'No se encontro {material.titulo} en {self.nombre}')
        return False 
    
    def mostrar_catalogo(self):
        print(f'Catalogo de la biblioteca "{self.nombre}":')
        if self.catalogo:
            for i, material in enumerate(self.catalogo, 1):
                print(f'{i}, {material}')
        else:
            print('La biblioteca no tiene materiales.') 

class Penalizacion:
    def __init__(self, usuario, dias_retraso):
        self.usuario = usuario
        self.dias_retraso = dias_retraso
        self.multa = dias_retraso * 1

    def __str__(self):
        return f"Penalización para {self.usuario.nombre}: {self.dias_retraso} días de retraso, multa de ${self.multa}"

In [11]:
libro1 = Libro("Cronicas del espacio", "Neil deGrasse Tyson", "Informativo")
revista1 = Revista("National Geographic", "Enero 2025", "Mensual")
digital1 = MaterialDigital("Data Sciencie for dummies", "PDF", "http://iakshs.com/python.pdf")

In [12]:
sucursal1 = Sucursal("Biblioteca Central")
sucursal2 = Sucursal("Biblioteca Sur")

In [13]:
usuario1 = Usuario("Pedro","Medina")
bibliotecario1 = Bibliotecario("Ana","Gonzalez", sucursal1)
penalizacion1 = Penalizacion(usuario1, 3)

In [14]:
bibliotecario1.agregar_material(libro1)
bibliotecario1.agregar_material(revista1)
bibliotecario1.agregar_material(digital1)

Cronicas del espacio fue agregado a la biblioteca Biblioteca Central
National Geographic fue agregado a la biblioteca Biblioteca Central
Data Sciencie for dummies fue agregado a la biblioteca Biblioteca Central


In [15]:
fecha_prestamo = datetime.now() - timedelta(days=10)
usuario1.pedir_prestado(libro1, fecha_prestamo)
print(penalizacion1)

Pedro pidio prestado 'Cronicas del espacio' el 2025-02-02, debe devolverlo antes del 2025-02-09
Penalización para Pedro: 3 días de retraso, multa de $3


In [16]:
usuario1.devolver_material(libro1)

Pedro ha devuelto 'Cronicas del espacio' con 3 dias de retraso. Penalizaciones acumuladas: 3


True

In [17]:
bibliotecario1.transferir_material(revista1, sucursal2)

El National Geographic fue eliminado de la biblioteca Biblioteca Central
National Geographic fue agregado a la biblioteca Biblioteca Sur
Ana transfirio "National Geographic" a "Biblioteca Sur
